Setup and loading InData class from a file

In [1]:
from indata_pro_ops import InData
import sqlalchemy as db
from sqlalchemy import inspect
import pandas as pd

## Accesing testing .json file (from server) and preprocessing it

In [2]:
# Temporary for testing purposes, preload test1.json
test1 = InData("test1.json")
test1.extract()
print(test1.username)
print(test1.open[1])

Tester_manual
2023-10-26 12:10:13


## Establishing connectrion with SQL server (MySQL localhost for testing purpose)

using pyodbc library

In [3]:
import pyodbc
# Establishing connection, using Devart ODBC Driver (Devart ODBC Driver for MySQL)
conn = pyodbc.connect('Driver={Devart ODBC Driver for MySQL};'
                      'User ID=root;'
                      'Password=legolas112;'
                      'Server=127.0.0.1;'
                      'Database=smokeout;'
                      'Port=3306;'
                      'Trusted_Connection=yes;')
# To perform any SQL QUERYs in pyodbc cursor() is needed
cursor = conn.cursor()
# Building QUERY strings:
# -------------------------------------------
#               WARNING!!!
# In MySQL syntax is different than in other SQL applications so for testing
# and final implementation there is need for different coding
# -------------------------------------------

# Inserting username into `user` table
username_insert = "INSERT INTO `smokeout`.`user` (`username`) VALUES ('"+test1.username+"');"
print(username_insert)
cursor.execute(username_insert)

# Commiting changes
#conn.commit()

INSERT INTO `smokeout`.`user` (`username`) VALUES ('Tester_manual');


using mysql.connector module

In [4]:
import mysql.connector as sql


db_connection = sql.connect(host='127.0.0.1', database='smokeout', 
user='root', password='legolas112')

df = pd.read_sql('SELECT * FROM table_name', con=db_connection)

C:\Users\01122464\AppData\Local\Temp\ipykernel_808\2619999710.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM table_name', con=db_connection)


DatabaseError: Execution failed on sql 'SELECT * FROM table_name': 1146 (42S02): Table 'smokeout.table_name' doesn't exist

In [ ]:
# Getting UserID from username - https://stackoverflow.com/questions/64565904/how-to-use-pd-read-sql-using-mysql-connector
UserID = pd.read_sql("SELECT UserID FROM use WHERE username="+test1.username, conn)
# Inserting opening and switching times
for i in test1.switch:
    switch_insert = "INSERT INTO `smokeout`.`user` (`username`) VALUES (`"+test1.username+"`);"

SQLAlchemy approach

In [9]:
# In SQLAlchemy (SQL module to work with pandas) to start connection
# first step is to create engine:
# -----------------------------------------------------------------
# For that although, the connection string is needed
conn_string = ''
# To connect to local database we need: user, password, port and name of database:
PWD = 'legolas112'
USR = 'root'
PORT = '3306'
DB = 'smokeout'
SQLALCHEMY_DATABASE_URI = 'mysql://{}:{}@localhost:{}/{}'.format(USR, PWD, PORT, DB)
conn_string = SQLALCHEMY_DATABASE_URI

engine = db.create_engine(conn_string)
conn = engine.connect()

# Extracting the metadata (needed to access the tables)
metadata = db.MetaData()
# Accessing respective tables: `user` and `inapp_user_data`
user = db.Table('user', metadata, autoload=True, autoload_with=engine)
inapp_user_data = db.Table('inapp_user_data', metadata, autoload=True, autoload_with=engine)
# printing that + also just columns
print(repr(metadata.tables['user']))
print(repr(metadata.tables['inapp_user_data']))
print(user.columns.keys())
# =====================================================================
# First operation is registering new user: in here that is covered as
# checking if username is already in database and if it is not
# inserting it in a new row
chkusr = False if conn.execute(db.select(user).where(user.c.username == test1.username)) is not None else True
try:
    start = 1/chkusr
    query = user.insert.values(username=test1.username)
    exe = conn.execute(query)
except:
    print("username already in database")

# =====================================================================
# Next operation is establishing UserID, with known username <from .json>
# To do that, run query: SELECT column `UserID` FROM table `user` 
# WHERE column `username` is equal to test1.username
sfwusr = conn.execute(db.select(user.c.UserID).where(user.c.username == test1.username))
UserID = sfwusr.fetchone() # fetchone(), fetchmany(n), fetchall() - printing a number of lines one/n-many/all
# The format of this data is not friendly:
print(UserID)
for i in UserID:
    UserID = i
    print(UserID)

# =====================================================================
# Finally, last operations are writing the inappuserdata into columns
# of approperiate table - `inapp_user_data`:
# Let's first create lists of values for INSERT queries
swt_list = []
for i in test1.switch:
    swt_list.append({"UserID": UserID, "SwitchPckg": i})
print(test1.open)

opn_list = []
for i in test1.open:
    opn_list.append({"UserID": UserID, "OpenPckg": i})
print(opn_list)
# Now we can create INSERT query:
query = inapp_user_data.insert()
exe_swt = conn.execute(query, swt_list)
exe_opn = conn.execute(query, opn_list)

Table('user', MetaData(), Column('UserID', INTEGER(), table=<user>, primary_key=True, nullable=False), Column('username', VARCHAR(collation='utf8mb4_polish_ci', length=30), table=<user>), Column('StartTime', DATETIME(), table=<user>), Column('PlanID', INTEGER(), ForeignKey('usage_plans.PlanID'), table=<user>), schema=None)
Table('inapp_user_data', MetaData(), Column('InDataID', INTEGER(), table=<inapp_user_data>, primary_key=True, nullable=False), Column('UserID', INTEGER(), ForeignKey('user.UserID'), table=<inapp_user_data>), Column('OpenPckg', DATETIME(), table=<inapp_user_data>), Column('SwitchPckg', DATETIME(), table=<inapp_user_data>), schema=None)
['UserID', 'username', 'StartTime', 'PlanID']
username already in database
(7,)
7
['2023-10-26 08:09:12', '2023-10-26 12:10:13', '2023-10-26 14:11:15', '2023-10-26 16:10:32', '2023-10-26 18:13:52', '2023-10-26 22:17:10']
[{'UserID': 7, 'OpenPckg': '2023-10-26 08:09:12'}, {'UserID': 7, 'OpenPckg': '2023-10-26 12:10:13'}, {'UserID': 7, 'O

In [12]:
# It's all nice, but it would be nice to first check if those inputs are not already in the Table:
swt_list = []
for i in test1.switch:
    try:
        chk_swt = False if conn.execute(db.select(inapp_user_data).where(inapp_user_data.c.SwitchPckg == i)) is not None else True
        start = 1/chk_swt
        swt_list.append({"UserID": UserID, "SwitchPckg": i})
    except:
        print("switching time already in database")

opn_list = []
for i in test1.open:
    try:
        chk_opn = False if conn.execute(db.select(inapp_user_data).where(inapp_user_data.c.OpenPckg == i)) is not None else True
        start = 1/chk_swt
        opn_list.append({"UserID": UserID, "OpenPckg": i})
    except:
        print("opening time already in database")
# Now we can create INSERT query:
query = inapp_user_data.insert()
exe_swt = conn.execute(query, swt_list)
exe_opn = conn.execute(query, opn_list)

switching time already in database
opening time already in database
opening time already in database
opening time already in database
opening time already in database
opening time already in database
opening time already in database


In [15]:
# Connecting to Azure
connAzure = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:smokeout.database.windows.net,1433;Database=SmokeOut_users;Uid=Krwawe;Pwd="+input("Password")+";Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
conn_str = f'mssql+pyodbc:///?odbc_connect={connAzure}'
engineAzure = db.create_engine(conn_str)